# Preprocessing: BJH046


This markdown files loads the clean data and does a bipolar rereference as well as epochs the data


In [1]:
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import re
import os
import mne
import IPython
import seaborn as sns

In [2]:
## Prep paths ##

subject = 'BJH046'
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}"
preproc_data_dir = f"/home/brooke/pacman/preprocessing/{subject}/ieeg"

In [3]:
## Load Data ##

# load filtered data #
filtered_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_notched_filtered_clean_ieeg.fif")

# load raw data #
raw_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_raw_clean_ieeg.fif")



Opening raw data file /home/brooke/pacman/raw_data/BJH046/ieeg/BJH046_notched_filtered_clean_ieeg.fif...
    Range : 0 ... 2085999 =      0.000 ...  1042.999 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/BJH046/ieeg/BJH046_notched_filtered_clean_ieeg-1.fif...
    Range : 2086000 ... 3140799 =   1043.000 ...  1570.399 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/BJH046/ieeg/BJH046_raw_clean_ieeg.fif...
    Range : 0 ... 2085999 =      0.000 ...  1042.999 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/BJH046/ieeg/BJH046_raw_clean_ieeg-1.fif...
    Range : 2086000 ... 3140799 =   1043.000 ...  1570.399 secs
Ready.


## Bipolar Rereferencing

In [4]:
# helper functions

def probe_and_num(elec_str): 
    ''' This convenience function takes an electrode string like LHH1 and outputs ('LHH', 1)
    '''
    if '_' in elec_str:
        return None, None
    else:   
        regex_str = '(\D+)(\d+)' # group of letters followed by group of digits
        matcher = re.compile(regex_str,re.IGNORECASE|re.DOTALL).search(elec_str)
        if matcher:
            probe, num = matcher.groups()
            return probe, int(num)
        else:
            return None, None
    
def find_bipolar_pair(ch, labels, remove):
    ''' Find the pair of a given electrode for bipolar referencing.
        Given a single *ch* and a list of *labels*, some of which you want to *remove*,
        this finds the next channel on that probe that isn't meant to be removed (WM is ok).
    '''
    bipolar_pair = None
    probe, num = probe_and_num(ch)
    other_in_probe = [other_ch for other_ch in labels if probe_and_num(other_ch)[0]==probe]
    for i in range(len(other_in_probe)-num):
        next_ch = probe + str(num+i+1)
        if next_ch in remove or next_ch == 'STI':
            continue
        else:
            bipolar_pair = next_ch
            break
    return bipolar_pair

In [5]:
# Here I iterate through the electrodes in my ROIs and match them up with their bipolar pair
pairs = []
anode = []
cathode = []
pairs_name = []
pairs_map = {}
remove = filtered_clean_fif.info['bads']
labels = filtered_clean_fif.info['ch_names']
for ch in labels:
    # if a channel is meant to be removed, it doesn't get to be in a bipolar pair
    if ch in remove or ch == 'STI':
        print(f"{ch} noref")
    else:
        pair = find_bipolar_pair(ch, labels, remove)
        if pair:
            anode.append(ch)
            cathode.append(pair)
            pairs.append((ch, pair))
            pairs_map[ch] = f"{ch}-{pair}"
            pairs_name.append(f"{ch}-{pair}")
            print(ch, pair)

AL1  AL2
AL2 AL3
AL3 AL4
AL4 AL5
REF1 noref
REF2 noref
AL5 AL6
AL6 AL7
AL7 AL8
AL8 AL9
AL9 AL10
AL10 AL11
AL11 AL12
BL1 BL2
BL2 BL3
BL3 BL4
BL4 BL5
BL5 BL8
BL6 noref
BL7 noref
CL1 CL2
CL2 CL3
CL3 CL4
CL4 CL5
CL5 CL6
CL6 CL7
CL7 CL8
CL8 CL9
CL9 CL10
CL10 CL11
CL11 CL12
CL12 CL13
CL13 CL14
CL14 CL15
CL15 CL16
CL16 CL17
CL17 CL18
DL1 DL2
DL2 DL3
DL3 DL4
DL4 DL5
DL5 DL6
DL6 DL7
DL7 DL8
DL8 DL9
DL9 DL10
DL10 DL11
DL11 DL12
EL1 EL2
EL2 EL3
EL3 EL4
EL4 EL5
EL5 EL6
EL6 EL7
EL7 EL8
EL8 EL9
EL9 EL10
EL10 EL11
EL11 EL12
FL1 FL2
FL2 FL3
FL3 FL4
FL4 FL5
FL5 FL6
FL6 FL7
FL7 FL8
FL8 FL9
FL9 FL10
FL10 FL11
FL11 FL12
FL12 FL13
FL13 FL14
FL14 FL15
GL1 GL2
GL2 GL3
GL3 GL4
GL4 GL5
GL5 GL6
GL6 GL7
GL7 GL8
GL8 GL9
GL9 GL10
GL10 GL11
GL11 GL12
HL1 HL2
HL2 HL3
HL3 HL4
HL4 HL5
HL5 HL6
HL6 HL7
HL7 HL8
HL8 HL9
HL9 HL10
HL10 HL11
HL11 HL12
IL1 IL2
IL2 IL3
IL3 IL4
IL4 IL5
IL5 IL6
IL6 IL7
IL7 IL8
IL8 IL9
IL9 IL10
IL10 IL11
IL11 IL12
JL1 JL2
JL2 JL3
JL3 JL4
JL4 JL5
JL5 JL6
JL6 JL7
JL7 JL8
JL8 JL9
JL9 JL10
JL10 JL11


In [6]:
## Apply Rereference #

if filtered_clean_fif.info['ch_names'] == raw_clean_fif.info['ch_names'] and filtered_clean_fif.info['bads'] == raw_clean_fif.info['bads']:
    
    # load filtered data
    filtered_clean_fif.load_data()
    
    # set filtered reference 
    bp_filt_fif = mne.set_bipolar_reference(filtered_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)
    
    # clear var
    bp_filt_fif = []
    filtered_clean_fif = []
    
    # load raw data
    raw_clean_fif.load_data()
    
    # set raw reference 
    bp_raw_fif = mne.set_bipolar_reference(raw_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)

    
    # clear var
    bp_raw_fif = []
    raw_clean_fif = []    
    
    

Reading 0 ... 3140799  =      0.000 ...  1570.399 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=195, n_times=3140800
    Range : 0 ... 3140799 =      0.000 ...  1570.399 secs
Ready.
Added the following bipolar channels:
AL1 -AL2, AL2-AL3, AL3-AL4, AL4-AL5, AL5-AL6, AL6-AL7, AL7-AL8, AL8-AL9, AL9-AL10, AL10-AL11, AL11-AL12, BL1-BL2, BL2-BL3, BL3-BL4, BL4-BL5, BL5-BL8, CL1-CL2, CL2-CL3, CL3-CL4, CL4-CL5, CL5-CL6, CL6-CL7, CL7-CL8, CL8-CL9, CL9-CL10, CL10-CL11, CL11-CL12, CL12-CL13, CL13-CL14, CL14-CL15, CL15-CL16, CL16-CL17, CL17-CL18, DL1-DL2, DL2-DL3, DL3-DL4, DL4-DL5, DL5-DL6, DL6-DL7, DL7-DL8, DL8-DL9, DL9-DL10, DL10-DL11, DL11-DL12, EL1-EL2, EL2-EL3, EL3-EL4, EL4-EL5, EL5-EL6, EL6-EL7, EL7-EL8, EL8-EL9, EL9-EL10, EL10-EL11, EL11-EL12, FL1-FL2, FL2-FL3, FL3-FL4, FL4-FL5, FL5-FL6, FL6-FL7, FL7-FL8, FL8-FL9, FL9-FL10, FL10-FL11, FL11-FL12, FL12-FL13, FL13-FL14, FL14-FL15, GL1-GL2, GL2-GL3, GL3-GL4, GL4-GL5, GL5-GL6, GL6-GL7, GL7-G

/tmp/ipykernel_5275/1336304392.py:12: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH046/ieeg/BJH046_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)


Closing /home/brooke/pacman/preprocessing/BJH046/ieeg/BJH046_bp_filtered_clean_data.fif
Writing /home/brooke/pacman/preprocessing/BJH046/ieeg/BJH046_bp_filtered_clean_data-1.fif
Closing /home/brooke/pacman/preprocessing/BJH046/ieeg/BJH046_bp_filtered_clean_data-1.fif
[done]
Reading 0 ... 3140799  =      0.000 ...  1570.399 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=195, n_times=3140800
    Range : 0 ... 3140799 =      0.000 ...  1570.399 secs
Ready.
Added the following bipolar channels:
AL1 -AL2, AL2-AL3, AL3-AL4, AL4-AL5, AL5-AL6, AL6-AL7, AL7-AL8, AL8-AL9, AL9-AL10, AL10-AL11, AL11-AL12, BL1-BL2, BL2-BL3, BL3-BL4, BL4-BL5, BL5-BL8, CL1-CL2, CL2-CL3, CL3-CL4, CL4-CL5, CL5-CL6, CL6-CL7, CL7-CL8, CL8-CL9, CL9-CL10, CL10-CL11, CL11-CL12, CL12-CL13, CL13-CL14, CL14-CL15, CL15-CL16, CL16-CL17, CL17-CL18, DL1-DL2, DL2-DL3, DL3-DL4, DL4-DL5, DL5-DL6, DL6-DL7, DL7-DL8, DL8-DL9, DL9-DL10, DL10-DL11, DL11-DL12, EL1-EL2, EL2-EL3, EL3-EL4

/tmp/ipykernel_5275/1336304392.py:25: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH046/ieeg/BJH046_bp_raw_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)


Closing /home/brooke/pacman/preprocessing/BJH046/ieeg/BJH046_bp_raw_clean_data.fif
Writing /home/brooke/pacman/preprocessing/BJH046/ieeg/BJH046_bp_raw_clean_data-1.fif
Closing /home/brooke/pacman/preprocessing/BJH046/ieeg/BJH046_bp_raw_clean_data-1.fif
[done]


In [ ]:
# Visualize it #

# bp_raw_fif.plot(events=events, color='b', bad_color = 'cyan', n_channels = 1, clipping = None, event_color = 'r')

## Epoching the data 

### Onset

In [7]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 1, tmin = -4, tmax = 12, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

Opening raw data file /home/brooke/pacman/preprocessing/BJH046/ieeg/BJH046_bp_filtered_clean_data.fif...
    Range : 0 ... 2253999 =      0.000 ...  1126.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH046/ieeg/BJH046_bp_filtered_clean_data-1.fif...
    Range : 2254000 ... 3140799 =   1127.000 ...  1570.399 secs
Ready.


/tmp/ipykernel_5275/1463149468.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH046/ieeg/BJH046_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


480 events found on stim channel STI
Event IDs: [0 1]
Not setting metadata
240 matching events found
No baseline correction applied
0 projection items activated
Loading data for 240 events and 32001 original time points ...


/tmp/ipykernel_5275/1463149468.py:18: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH046/ieeg/BJH046_bp_clean_pres-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


1 bad epochs dropped
Loading data for 1 events and 32001 original time points ...
Splitting into 3 parts
Loading data for 80 events and 32001 original time points ...
Loading data for 80 events and 32001 original time points ...
Loading data for 79 events and 32001 original time points ...


### Trial End

In [8]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 0, tmin = -5, tmax = 5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

Opening raw data file /home/brooke/pacman/preprocessing/BJH046/ieeg/BJH046_bp_filtered_clean_data.fif...
    Range : 0 ... 2253999 =      0.000 ...  1126.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH046/ieeg/BJH046_bp_filtered_clean_data-1.fif...
    Range : 2254000 ... 3140799 =   1127.000 ...  1570.399 secs
Ready.


/tmp/ipykernel_5275/749539899.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH046/ieeg/BJH046_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


480 events found on stim channel STI
Event IDs: [0 1]
Not setting metadata
240 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 20001 original time points ...
Splitting into 2 parts
Loading data for 120 events and 20001 original time points ...


/tmp/ipykernel_5275/749539899.py:18: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH046/ieeg/BJH046_bp_clean_end-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


Loading data for 120 events and 20001 original time points ...


### Last Away

In [9]:
## Load Behavioral Data ##
last_away_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_last_away_events.csv")
last_away_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,19,33400,0,1
1,1,18,51700,0,1
2,2,3,64000,0,1
3,3,7,71500,0,1
4,7,12,106600,0,1
...,...,...,...,...,...
204,234,16,2857200,0,1
205,235,10,2866900,0,1
206,236,14,2878400,0,1
207,237,7,2890300,0,1


In [10]:
# create events
last_away_events = last_away_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [11]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
last_away_epochs = mne.Epochs(bp_filtered_fif, last_away_events, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH046/ieeg/BJH046_bp_filtered_clean_data.fif...
    Range : 0 ... 2253999 =      0.000 ...  1126.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH046/ieeg/BJH046_bp_filtered_clean_data-1.fif...
    Range : 2254000 ... 3140799 =   1127.000 ...  1570.399 secs
Ready.
Not setting metadata
209 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 20001 original time points ...
Splitting into 2 parts
Loading data for 105 events and 20001 original time points ...


/tmp/ipykernel_5275/678285560.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH046/ieeg/BJH046_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_5275/678285560.py:13: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH046/ieeg/BJH046_bp_filtered_clean_last_away_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


Loading data for 104 events and 20001 original time points ...


### First Dot

In [12]:
## Load Behavioral Data ##
first_dot_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_dot_events.csv")
first_dot_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,19,21900,0,1
1,1,18,47600,0,1
2,2,3,61800,0,1
3,4,11,79600,0,1
4,6,6,96100,0,1
...,...,...,...,...,...
223,235,10,2865500,0,1
224,236,14,2876200,0,1
225,237,7,2888600,0,1
226,238,15,2899700,0,1


In [13]:
# create events
first_dot_data = first_dot_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [14]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_dot_epochs = mne.Epochs(bp_filtered_fif, first_dot_data, 
                          event_id = 1, tmin = -3, tmax =6, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH046/ieeg/BJH046_bp_filtered_clean_data.fif...
    Range : 0 ... 2253999 =      0.000 ...  1126.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH046/ieeg/BJH046_bp_filtered_clean_data-1.fif...
    Range : 2254000 ... 3140799 =   1127.000 ...  1570.399 secs
Ready.
Not setting metadata
228 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 18001 original time points ...
Splitting into 2 parts
Loading data for 114 events and 18001 original time points ...


/tmp/ipykernel_5275/4126832509.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH046/ieeg/BJH046_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_5275/4126832509.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH046/ieeg/BJH046_bp_filtered_clean_first_dot_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif", overwrite = True, split_size = '1.9GB')


Loading data for 114 events and 18001 original time points ...


## First Move

In [15]:
## Load Behavioral Data ##
first_move_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_move_events.csv")
first_move_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,19,21100,0,1
1,1,18,44700,0,1
2,2,3,61200,0,1
3,3,7,71500,0,1
4,4,11,76200,0,1
...,...,...,...,...,...
230,235,10,2864600,0,1
231,236,14,2875400,0,1
232,237,7,2887900,0,1
233,238,15,2899600,0,1


In [16]:
# create events
first_move_data = first_move_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [17]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_move_epochs = mne.Epochs(bp_filtered_fif, first_move_data, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_move_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_move_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH046/ieeg/BJH046_bp_filtered_clean_data.fif...
    Range : 0 ... 2253999 =      0.000 ...  1126.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH046/ieeg/BJH046_bp_filtered_clean_data-1.fif...
    Range : 2254000 ... 3140799 =   1127.000 ...  1570.399 secs
Ready.
Not setting metadata
235 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 20001 original time points ...
Splitting into 2 parts
Loading data for 118 events and 20001 original time points ...


/tmp/ipykernel_5275/3347570552.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH046/ieeg/BJH046_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_5275/3347570552.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH046/ieeg/BJH046_bp_filtered_clean_first_move_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_move_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_move_events.fif", overwrite = True, split_size = '1.9GB')


Loading data for 117 events and 20001 original time points ...


## Ghost Attack

In [18]:
## Load Behavioral Data ##
ghost_attack_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_attack_events.csv")
ghost_attack_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,4,11,81300,0,1
1,6,6,96600,0,1
2,10,1,142200,0,1
3,12,13,179800,0,1
4,16,16,227700,0,1
5,18,15,246300,0,1
6,22,6,296200,0,1
7,28,11,353200,0,1
8,41,11,511900,0,1
9,42,14,523900,0,1


In [19]:
# create events
ghost_attack_data = ghost_attack_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [20]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
ghost_attack_epochs = mne.Epochs(bp_filtered_fif, ghost_attack_data, 
                          event_id = 1, tmin = -4, tmax =6, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
ghost_attack_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_ghost_attack_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH046/ieeg/BJH046_bp_filtered_clean_data.fif...
    Range : 0 ... 2253999 =      0.000 ...  1126.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH046/ieeg/BJH046_bp_filtered_clean_data-1.fif...
    Range : 2254000 ... 3140799 =   1127.000 ...  1570.399 secs
Ready.
Not setting metadata
49 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 20001 original time points ...
Loading data for 49 events and 20001 original time points ...


/tmp/ipykernel_5275/2681655913.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH046/ieeg/BJH046_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_5275/2681655913.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH046/ieeg/BJH046_bp_filtered_clean_ghost_attack_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  ghost_attack_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_ghost_attack_events.fif", overwrite = True, split_size = '1.9GB')
